# <h1 align="center">Práctica 8. Grafos</h1>
<h3 style="display:block; margin-top:5px;" align="center">Estructuras de datos</h3>
<h3 style="display:block; margin-top:5px;" align="center">Grado en Ciencia de Datos</h3>
<h3 style="display:block; margin-top:5px;" align="center">2023-2024</h3>    
<h3 style="display:block; margin-top:5px;" align="center">Universitat Politècnica de València</h3>
<br>

**Pon/poned aquí tú/vuestros nombre(s):**
- Pablo Pertusa Canales

## Índice
1. ### [Introducción: Objetivos de la práctica](#intro)
1. ### [Actividad 1: Carga de grafos en un formato compacto](#act1)
1. ### [Actividad 2: Obtener el camino más corto con Dijkstra a un conjunto de destinos](#act2)

<a id='intro'></a>
## Introducción: Objetivos de la práctica

Esta práctica sobre el tema de grafos está centrada en la obtención del camino más corto desde un vértice origen a un conjunto de destinos. Se trata, por tanto, de un problema de caminos más cortos del tipo ["single source"](https://en.wikipedia.org/wiki/Shortest_path_problem#Single-source_shortest_paths).

Al tratarse de grafos con pesos positivos, vamos a utilizar el [algoritmo de Dijkstra](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm).

Utilizaremos un grafo procedente de la *9th DIMACS Implementation Challenge - Shortest Paths* disponible a traves [del siguiente enlace](https://www.diag.uniroma1.it/challenge9/) que hemos procesado para convertirlo a un formato más fácil de cargar de manera compacta.

<a id='act1'></a>
## Actividad 1: Carga de grafos un formato más compacto

La primera actividad consiste en completar el constructor de la clase `CompactDiGraph` cuyo constructor recibe como parámetro el nombre de un fichero en un formato que describimos a continuación.

El objetivo es poder cargar grafos pensando en que ocupen menos que con las listas de adyacencia vistas en clase pero que, una vez cargados, no vamos a poder modificar (al menos fácil o eficientemente).

El formato propuesto sigue siendo una lista de adyacencia pero:

- Todos los vértices están identificados por un número entero entre `0` y `self.nV-1` donde `self.nV` es el número de vértices.
- Todas las aristas están guardadas en dos únicos vectores `numpy` de tamaño `self.nA`:

    - `self.Adest` guarda los vértices destino de cada arista.
    - `self.Aweight` guarda los pesos de dichas aristas.
    
    Ambos vectores han de crearse de tipo `np.int32`. Por ejemplo, para crear un vector de `n` elementos de tipo `np.int32` puedes hacer:
    
    ```
    np.zeros(n, dtype=np.int32)
    ```
    
- Para poder saber cuáles son las aristas que salen de cada vértice debemos tener en cuenta:

    - Las aristas están ordenadas por el vértice origen, por lo que todas las aristas que salen de cada vértice son contiguas.
    - Hay un atributo llamado `self.vertexIndex` de tamaño `self.nV+1` (observa el `+1`) que, en la posición `u` contiene el índice de la primera arista que sale de `u` en los vectores `self.Adest` y `self.Aweight`.
    
Vamos a poner un ejemplo para que quede todo más claro. El siguiente grafo corresponde al ejemplo de los apuntes de teoría:

![(el grafo de los apuntes de teoría)](dijkstra.svg)

Vemos que este grafo tiene 5 vértices (s, u, v, x, y) que ahora se renombranán, respectivamente: 0,1,2,3,4.

Este grafo se representaría de la manera siguiente:

```
5 10
# Out degrees:
2
2
1
3
2
# Edges:
0 1 10
0 3 5
1 2 1
1 3 2
2 4 4
3 1 3
3 2 9
3 4 2
4 2 6
4 0 7
```

Analicemos el fichero, la primera línea nos dice el número de vértices y el número total de aristas del grafo:

```
5 10
```

Esos 5 vértices se van a numerar 0,1,2,3 y 4.

A continuación vendría el grado de salida de cada vértice:

- s ahora es 0 y tiene grado de salida 2.
- u ahora es 1 y tiene grado de salida 2.
- v ahora es 2 y tiene grado de salida 1.
- x ahora es 3 y tiene grado de salida 3.
- y ahora es 4 y tiene grado de salida 2.

Finalmente aparecen 10 líneas con todas las aristas, ordenadas por el vértice origen, observa que viendo la primera columna también se podría inferir el grado de salida de cada vértice (la información previa es redundante pero simplifica la lectura):

```
0 1 10
0 3 5
1 2 1
1 3 2
2 4 4
3 1 3
3 2 9
3 4 2
4 2 6
4 0 7
```

esto significa que, en nuestro ejemplo, tenemos los siguientes datos:

- `self.nV = 5`
- `self.nA = 10`
- `self.Adest = np.array([1,3,2,3,4,1,2,4,2,0],dtype=np.int32)`
- `self.Aweight = np.array([10,5,1,2,4,3,9,2,6,7], dtype=np.int32)`

Observa que `self.Adest` y `self.Aweight` son, respectivamente, la segunda y tercera columna de la parte de datos de las aristas.

Para poder a las aristas que salen de un vértice dado necesitamos además el atributo self.vertexIndex que nos indica el índice donde está la primera arista de un vértice dado:

- `self.vertexIndex = np.array([0, 2, 4, 5, 8, 10], dtype=np.int32)`

Por ejemplo, para recorrer las aristas que salen del vértice 3 tenemos que recorrer `self.Adest`y `self.Aweight` en el rango de 5 a 8 no inclusive (es decir, utilizar `range(5,8)`).

Observa que para cualquier vértice `u` podemos hacer `range(self.vertexIndex[u], self.vertexIndex[u+1])` ya que:

- funciona incluso si un vértice es de tipo sumidero (no tiene aristas de salida) ya que el valor de `self.vertexIndex` del siguiente vértice será el mismo y si haces, pongamos, `range(7,7)` en un bucle éste hará 0 iteraciones.
- funciona también para el último vértice, por eso el vector `self.vertexIndex` es de longitud `self.nV+1`.

Es fácil construir el vector `self.vertexIndex` mientras leemos los grados de salida del fichero, basta con ir sumando de manera acumulativa los grados de salida:

vértice | grado salida | vertexIndex
--------|--------------|------------
   0    |      2       |     0
   1    |      2       |     2 = 0+2 
   2    |      1       |     4 = 2+2
   3    |      3       |     5 = 4+1
   4    |      2       |     8 = 5+3
   NO   |     NO       |     10 = 8+2


In [24]:
import numpy as np

class CompactDiGraph:
    
    def __init__(self, filename):
        with open(filename) as f:
            nv,na = f.readline().split()
            self.nV = int(nv)
            self.nA = int(na)
            self.vertexIndex = np.zeros(self.nV+1, dtype=np.int32)
            self.Adest   = np.zeros(self.nA, dtype=np.int32)
            self.Aweight = np.zeros(self.nA, dtype=np.int32)
            accum = 0 # para ir calculado vertexIndex
            f.readline() # lee # Out degrees:
            for i in range(self.nV):
                 grado = int(f.readline())
                 accum += grado
                 self.vertexIndex[i+1] = accum
            f.readline()
            for i in range(self.nA):
                 x,dest,peso = f.readline().split()
                 self.Adest[i] = dest
                 self.Aweight[i] = peso
            
                 
                
    def edges_from(self, u):
        for i in range(self.vertexIndex[u], self.vertexIndex[u+1]):
            yield self.Adest[i], self.Aweight[i]
            
    def __repr__(self):
        resul = []
        resul.append(f'nV = {self.nV}')
        resul.append(f'nA = {self.nA}')
        for u in range(self.nV):
            resul.append(f'Vertex {u}:')
            for v,w in self.edges_from(u):
                resul.append(f'{u} {v} {w}')
        return '\n'.join(resul)                          

Para comprobar el correcto funcionamiento del código anterior puedes probar a cargar el fichero `ejemplo.grafo` que proporcionamos en la práctica:

In [25]:
grafo = CompactDiGraph('ejemplo.grafo')
print(grafo)

nV = 5
nA = 10
Vertex 0:
0 1 10
0 3 5
Vertex 1:
1 2 1
1 3 2
Vertex 2:
2 4 4
Vertex 3:
3 1 3
3 2 9
3 4 2
Vertex 4:
4 2 6
4 0 7


Y ver que da lo siguiente:
    
```
nV = 5
nA = 10
Vertex 0:
0 1 10
0 3 5
Vertex 1:
1 2 1
1 3 2
Vertex 2:
2 4 4
Vertex 3:
3 1 3
3 2 9
3 4 2
Vertex 4:
4 2 6
4 0 7
```

<a id='act2'></a>
## Actividad 2: Obtener el camino más corto con Dijkstra a un conjunto de destinos

En esta actividad debes modificar ligeramente el código de Dijkstra proporcionado en los apuntes de teoría, y que copiamos a continuación para más comodidad:

```python
    def Dijkstra(self, src):
        dist = { src:0 } # asocia una distancia a cada vértice
        pred = {} # para recuperar el camino de menor peso
        fixed = set() # cjt de vértices que ya han sido fijados
        eligible = { src } # cjt con src como elemento inicial
        iter = 0
        while len(eligible)>0:
            d,u = min( (dist[v],v) for v in eligible)
            eligible.remove(u) # lo sacamos de eligible
            fixed.add(u)       # y lo añadimos a fixed
            for v,weight in self._adj[u]: # actualizamos la cota
                if v not in fixed:        # de los adyacentes a v
                    if v not in dist:
                        pred[v], dist[v] = u, dist[u] + weight
                        eligible.add(v)
                    elif dist[v] > dist[u] + weight:
                        pred[v], dist[v] = u, dist[u] + weight
            iter += 1
        return dist, pred
```

Se piden las siguientes modificaciones:

- Adaptarlo a la nueva representación de la clase `CompactDiGraph` donde ya no existe `self._adj`.

- En la cabecera, además de recibir un vértice origen `src` se proporciona una lista de vértices destinos:

```python
    def Dijkstra(self, src, dsts):
        ...
```

- El método ya no debe devolver los diccionarios Python `dist` y `pred`. En su lugar devolverá un diccionario Python que asocia, a cada vértice destino (los elementos de la lista `dsts`) una tupla con estos 2 campos:

    1. El coste del camino desde `src` al destino.
    2. El camino dado como una lista de vértices desde el origen al destino de ese camino.
    
Para que la función `Dijkstra` sea algo más eficiente, el bucle principal (`while len(eligible)>0:`) no debe ejecutarse siempre hasta el final sino que el algoritmo deberá parar tan pronto como todos los vértices de `dsts` estén fijados. Para ello puedes tener una variable de tipo conjunto `destinos = set(dsts)` y cada vez que fijes un vértice, si el vértice fijado pertenece a `destinos` lo eliminas de él (método `remove`). Basta con añadir una condición más en el `while`.

Para recuperar el camino desde `src` a cada vértice de `dsts` se debe recuperar el camino utilizando el diccionario `pred`. Como esto nos da el camino en sentido inverso, hace falta darle la vuelta (utilizando `reverse`, por ejemplo).

Puedes completar el código del método `Dijkstra` en la siguiente clase Python que hereda de la del ejercicio anterior donde has implementado el constructor de la clase `CompactDiGraph`:

In [42]:
class CompactDiGraph2(CompactDiGraph):
    
    # MODIFICAR TAL Y COMO SE EXPLICA/PIDE ARRIBA
    def Dijkstra(self, src, dsts):
        dist = { src:0 } # asocia una distancia a cada vértice
        pred = {} # para recuperar el camino de menor peso
        fixed = set() # cjt de vértices que ya han sido fijados
        eligible = { src } # cjt con src como elemento inicial
        iter = 0
        destinos = set(dsts)
        while len(eligible)>0:
            d,u = min( (dist[v],v) for v in eligible)
            eligible.remove(u) # lo sacamos de eligible
            fixed.add(u)      # y lo añadimos a fixed
            for i in range(self.vertexIndex[u], self.vertexIndex[u+1]):
                v = self.Adest[i]
                if v not in fixed:        
                    if v not in dist:
                        pred[v], dist[v] = u, dist[u] + self.Aweight[i]
                        eligible.add(v)
                    elif dist[v] > dist[u] + self.Aweight[v]:
                        pred[v], dist[v] = u, dist[u] + self.Aweight[i]
            iter += 1
            if u in destinos:
                destinos.remove(u)
            if len(destinos) == 0:
                break
        
        resul = {}
        for i in dsts:
            camino = [i]
            x = pred[i]
            camino.append(x)
            while x != src:
                x = pred[x]
                camino.append(x)
            
            camino = list(reversed(camino))
            resul[i] = (dist[i], camino)
        
        print(iter, 'iteraciones')
        return resul
            

Una vez hayas modificado adecuadamente el método `Dijkstra` en la celda anterior puedes comprobar su correcto funcionamiento cargando el grafo `rome99.grafo` que hemos dejado en la carpeta de la práctica y que es una adaptación a nuestro formato del fichero del siguiente enlace: 

[http://users.diag.uniroma1.it/challenge9/download.shtml](http://users.diag.uniroma1.it/challenge9/download.shtml)

La descripción de este grafo es la siguiente:

> Large portion of the directed road network of the city of Rome, Italy, from 1999. The graph contains 3353 vertices and 8870 edges. Vertices correspond to intersections between roads and edges correspond to roads or road segments. The file is given in the standard Challenge 9 format.

Para comprobar el correcto funcionamiento del método Dijkstra, una vez cargado un `DiGraphActividad2` llamado `grafo` con el fichero `rome99.grafo` la llamada:

```python
resul = grafo.Dijkstra(9, [49, 79, 119])
print(resul)
```

Nos debería devolver un diccionario Python similar al siguiente:

```
{49: (19340, [9, 8, 10, 150, 151, 145, 62, 60, 61, 54, 55, 49]), 79: (26464, [9, 8, 10, 150, 151, 145, 62, 60, 61, 65, 120, 119, 123, 124, 125, 127, 126, 72, 86, 85, 79]), 119: (20358, [9, 8, 10, 150, 151, 145, 62, 60, 61, 65, 120, 119])}
```

Si contamos el número de iteraciones del bucle `while` y lo mostramos veremos que da un total de 555 iteraciones, mucho menos que el número de vértices de dicho grafo (3353).


> **ATENCIÓN:** es posible que el camino más corto entre 2 vértices difiera en la secuencia de vértices si hay más de un camino más corto posible, por lo que puede que tu solución sea diferente y esté bien. Lo que no debería ser válido es una longitud diferente para dicho camino.

> **NOTA:** en esta práctica nos hemos limitado a modificar muy ligeramente el algoritmo de Dijkstra visto en clase. Esta versión utiliza un diccionario Python para guardar las distancias y la función `min` para localizar la menor de entre los vértices `eligible`. Sería  más eficiente utilizar un *min-heap* para mantener en todo momento el vértice eligible de menor distancia, pero esto requiere utilizar un *min-heap doblemente indexado* (poder localizar un elemento dentro del *heap* a partir de su clave) y esto complicaría bastante su implementación. Esta mejora no se exige ni va para examen ni se ha explicado en clase, pero el que tenga interés puede buscarla en la bibliografía.

In [43]:
grafo = CompactDiGraph2('rome99.grafo')
resul = grafo.Dijkstra(9, [49, 79, 119])
print(resul)

243 iteraciones
{49: (19340, [9, 8, 10, 150, 151, 145, 62, 60, 61, 54, 55, 49]), 79: (26464, [9, 8, 10, 150, 151, 145, 62, 60, 61, 65, 120, 119, 123, 124, 125, 127, 126, 72, 86, 85, 79]), 119: (20358, [9, 8, 10, 150, 151, 145, 62, 60, 61, 65, 120, 119])}


El resultado anterior debería de ser parecido a esto:

```
555 iteraciones
{49: (19340, [9, 8, 10, 150, 151, 145, 62, 60, 61, 54, 55, 49]), 79: (26464, [9, 8, 10, 150, 151, 145, 62, 60, 61, 65, 120, 119, 123, 124, 125, 127, 126, 72, 86, 85, 79]), 119: (20358, [9, 8, 10, 150, 151, 145, 62, 60, 61, 65, 120, 119])}
```